In [37]:
import pandas as pd
import numpy as np
import os
import pickle
import torch
import string
from functools import reduce
pd.set_option('display.max_colwidth', 500)

import tqdm

In [8]:
graph_df = pd.read_csv("../data/neeg/data2.csv", names=["graph_id", "word_id", "score"], sep=' ')
word_encodings_df = pd.read_csv("../data/neeg/encoding_with_args.csv", names=["encoding", "word"], sep=' ')

In [9]:
graph_df.head(5)

,graph_id,word_id,score
0,0,14540,0.090909
1,0,21575,0.090909
2,0,23798,0.090909
3,0,46676,0.090909
4,0,70381,0.090909


In [10]:
word_encodings_df = word_encodings_df.set_index('encoding')

In [11]:
word_encodings_df.head(5)

,word
encoding,
0,--including_obj
1,--including_subj
2,70s_obj
3,70s_subj
4,ACT_prep_on


In [12]:
graph_df.merge(word_encodings_df, left_on="word_id", right_on="encoding", how='left')

,graph_id,word_id,score,encoding,word
0,0,14540,0.090909,14540,announced_subj
1,0,21575,0.090909,21575,burdened_subj
2,0,23798,0.090909,23798,charged_obj
3,0,46676,0.090909,46676,giving_subj
4,0,70381,0.090909,70381,produced_subj
5,0,77369,0.090909,77369,replaced_obj
6,0,80777,0.181818,80777,s_subj
7,0,82944,0.090909,82944,sell_subj
8,0,88537,0.090909,88537,spent_subj
9,0,104581,0.090909,104581,wrote_subj


In [13]:
with open("../data/neeg/corpus_index_train0_with_args_all.data", 'rb') as f:
    data = pickle.load(f)

In [14]:
unk, tokens, corr = data

In [15]:
df = pd.DataFrame.from_dict({'embeddings':tokens, 'correct':corr})

In [16]:
df['np_embeddings'] = df.embeddings.apply(np.array)
df['sents'] = df.np_embeddings.apply(lambda s: list(map(lambda x: word_encodings_df.loc[x].word if x in word_encodings_df.index else '', s)))
df['sents'] = df['sents'].apply(lambda s: ['' if isinstance(x, float) or x is None else x for x in s])

In [44]:
def readable_chain(chain_list, pretty_print=False):
    #correspondences = zip(chain_list[:13], chain_list[13:26], chain_list[26:39], chain_list[39:52])
    #Convert from V S O P to S V O P
    quadruples = zip(chain_list[13:26], chain_list[:13], chain_list[26:39], chain_list[39:52])
    sents = [quad_to_clean_sents(qd) for qd in quadruples]
    chain = sents[:-5]
    candidates = sents[-5:]
    if pretty_print:
        for i, s in enumerate(chain):
            s = filter(lambda q: q != '', s)
            print(i, ' '.join(s))
        print('')
        for a, s in zip(string.ascii_lowercase, candidates):
            s = filter(lambda q: q != '', s)
            print("%s) %s"%(a, ' '.join(s)))
    return {'chain':chain, 'candidates':candidates}    
def quad_to_clean_sents(quadruple):
    quadruple = ['' if w == '--including_obj' else w for w in quadruple]
    quadruple = [w.split('_subj')[0] if w.endswith('_subj')  else w for w in quadruple]
    quadruple = [w.split('_obj')[0] if w.endswith('_obj')  else w for w in quadruple]
    quadruple = [w.replace('+', ' ') for w in quadruple]
    quadruple = [w.replace('+', ' ') for w in quadruple]
    quadruple = [w.replace('_prep_', ' ') for w in quadruple]
    #quadruple = filter(lambda q: q != '', quadruple)
    return quadruple
    
    
with open('dataset/neeg.pickle', 'wb') as f:
    for i in tqdm.trange(len(df.index)):
        chain_dict = readable_chain(df.sents.loc[i])
        chain_dict['correct'] = int(df.correct.loc[i])
        pickle.dump(chain_dict, f)

100%|██████████| 140331/140331 [00:20<00:00, 6841.29it/s]
